# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `react_COthin` network as an example, which contains 37 species and 287 reactions for carbon-oxygen chemistry.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **test_chemistry.cpp**: Test program with multiple validation scenarios
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs and follows the same patterns as the odesolve_kokkos example.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [ ]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the react_COthin network
network = Network('../networks/react_COthin')

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [ ]:
# Show the first few species
print("First 10 species:")
for i, species in enumerate(network.species[:10]):
    print(f"  {i}: {species.name} (mass: {species.mass:.2f}, charge: {species.charge})")

if len(network.species) > 10:
    print(f"  ... and {len(network.species) - 10} more species")

print(f"\nTotal species: {len(network.species)}")

In [ ]:
# Show a few example reactions
print("First 5 reactions:")
for i, reaction in enumerate(network.reactions[:5]):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [ ]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print("Generated files are located in the builds directory.")

## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [ ]:
# List generated files
builds_dir = "../src/jaff/builds"
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

In [ ]:
# Show the header file structure
with open("../src/jaff/builds/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

print("chemistry_ode.hpp (first 30 lines):")
print("=" * 50)
for i, line in enumerate(lines[:30]):
    print(f"{i+1:2d}: {line.rstrip()}")
    
if len(lines) > 30:
    print(f"... and {len(lines) - 30} more lines")

In [ ]:
# Show the CMakeLists.txt structure
with open("../src/jaff/builds/CMakeLists.txt", 'r') as f:
    content = f.read()

print("CMakeLists.txt:")
print("=" * 50)
print(content)

## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [ ]:
# Show the ChemistryODE class definition
with open("../src/jaff/builds/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, min(class_start + 20, len(lines))):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

In [ ]:
# Show main function structure from chemistry_ode.cpp
with open("../src/jaff/builds/chemistry_ode.cpp", 'r') as f:
    lines = f.readlines()

# Find main function
main_start = None
for i, line in enumerate(lines):
    if "int main(" in line:
        main_start = i
        break

if main_start:
    print("Main function structure (first 25 lines):")
    print("=" * 50)
    for i in range(main_start, min(main_start + 25, len(lines))):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")

## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

In [ ]:
print("""To build and run the generated C++ chemistry solver:

1. Copy the generated files to your desired directory:
   cp ../src/jaff/builds/* /path/to/your/chemistry_project/

2. Configure the build with CMake:
   cd /path/to/your/chemistry_project/
   cmake -S . -B build

3. Build with parallel compilation:
   cmake --build build -j

4. Run the chemistry solver:
   ./build/chemistry_ode
   
   # Or with custom time interval (100 years in seconds):
   ./build/chemistry_ode 3.15576e9

5. Run the test suite:
   ./build/test_chemistry
   
   # Or via CTest:
   cd build && ctest

The solver uses the Kokkos-kernels BDF solver which is well-suited for
stiff chemical reaction networks. It automatically adapts time steps
and uses numerical differentiation for the Jacobian.""")

## Step 7: Solver Configuration Options

The generated solver includes several configuration options:

In [ ]:
print("""Solver Configuration:

Time Integration:
- Default end time: 100 years (3.15576e9 seconds)
- Initial time step: 1.0e-10 seconds (adaptive)
- Maximum time step: (t_end - t_start) / 100

Initial Conditions:
- Default: All species start at 1.0e-10 except first species (1.0e-6)
- Can be loaded from file: ./chemistry_ode end_time initial_conditions.txt
- Can save results to file: ./chemistry_ode end_time input.txt output.txt

BDF Solver Features:
- Backward Differentiation Formula for stiff ODEs
- Adaptive time stepping
- Numerical Jacobian (set analytical Jacobian to zero)
- Kokkos parallel execution on default execution space

Output:
- Prints number of species and reactions
- Shows initial and final concentrations (first 10 species)
- Performs conservation check (sum of all species)
- Reports Kokkos execution space used""")

## Step 8: Test Program Features

The generated test program includes comprehensive validation:

In [ ]:
# Show test program structure
with open("../src/jaff/builds/test_chemistry.cpp", 'r') as f:
    content = f.read()

# Count test cases
test_count = content.count('Test ')
print(f"Test program includes {test_count} test cases:")
print()

# Extract test descriptions
lines = content.split('\n')
for i, line in enumerate(lines):
    if 'Test ' in line and ':' in line:
        test_name = line.strip().replace('std::cout << "', '').replace('";', '')
        print(f"• {test_name}")
        
        # Look for the next few lines to get test description
        for j in range(i+1, min(i+5, len(lines))):
            if 'const scalar_type t_end' in lines[j]:
                time_info = lines[j].strip()
                if 'day' in time_info:
                    print(f"  - Duration: 1 day")
                elif 'month' in time_info:
                    print(f"  - Duration: ~1 month") 
                elif 'year' in time_info:
                    print(f"  - Duration: 1 year")
                break
        print()

print("""Each test validates different aspects:
- Finite solutions (no NaN/Inf values)
- Mass conservation (within tolerance)
- Positivity (no negative concentrations)
- Reasonableness (concentrations < 1.0)""")

## Summary

This notebook demonstrated how to:

1. **Load a chemical network** using JAFF's Network class
2. **Generate C++ Kokkos code** using the Builder with the `kokkos_ode` template
3. **Examine the generated files** and understand their structure
4. **Configure and run** the chemistry solver with different options
5. **Validate results** using the comprehensive test suite

The generated C++ code provides a high-performance chemistry solver that:
- Uses Kokkos for portable parallelism
- Employs Kokkos-kernels BDF solver for stiff ODEs
- Handles complex chemical reaction networks efficiently
- Includes comprehensive testing and validation
- Supports customizable initial conditions and output

This workflow enables researchers to quickly generate optimized chemistry solvers from reaction network definitions, suitable for integration into larger astrophysical simulation codes.